<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/14_Reasoning.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 14. Reasoning

## Goal
Apply reasoning and CoT in the prompt.

In [1]:
! pip3 install -qU langchain-upstage requests python-dotenv datasets arize-phoenix

## UPSTAGE_API_KEY
To obtain your Upstage API key, follow these steps:

1. Visit the Upstage AI console at <https://console.upstage.ai>.
2. Sign up for an account if you don't already have one.
3. Log in to your account.
4. Navigate to the API key section.
5. Generate your API key.
6. Copy the key and save it securely.

![Console](./figures/console.upstage.ai.jpg)

In [2]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [3]:
import phoenix as px

ssession = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
from phoenix.trace.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument

<bound method LangChainInstrumentor.instrument of <phoenix.trace.langchain.instrumentor.LangChainInstrumentor object at 0x104c71be0>>

In [5]:
from datasets import load_dataset

ds = load_dataset("SkunkworksAI/reasoning-0.01")

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
        num_rows: 29857
    })
})

In [7]:
# DatasetDict({
#    train: Dataset({
#        features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
#        num_rows: 29857
#    })
# })

# Let's take a look at the first five examples in the training set
# Skip Output: {example['output']}
reasoning_chain_examples = ""
for i, example in enumerate(ds["train"].select(range(2))):
    reasoning_chain_examples += f"""----
Example {i + 1}:

Instruction: {example['instruction']}

Reasoning: {example['reasoning']}

Reasoning Chains: {example['reasoning_chains']}
"""

pprint(reasoning_chain_examples)

('----\n'
 'Example 1:\n'
 '\n'
 'Instruction: If a die is rolled three times, what is the probability of '
 'getting a sum of 11? None\n'
 '\n'
 'Reasoning: 1. Understand the problem: We need to find the probability of '
 'getting a sum of 11 when rolling a die three times.\n'
 '2. Calculate total possible outcomes: A die has 6 faces, so for each roll, '
 'there are 6 possibilities. For three rolls, the total possible outcomes are '
 '6^3 = 216.\n'
 '3. Identify favorable outcomes: List all combinations of rolls that result '
 'in a sum of 11. There are 18 such combinations.\n'
 '4. Calculate probability: Divide the number of favorable outcomes by the '
 'total possible outcomes: 18 / 216 = 1/12.\n'
 '5. Conclusion: The probability of getting a sum of 11 when rolling a die '
 'three times is 1/12.\n'
 '\n'
 "Reasoning Chains: [{'step': 1, 'thought': 'Understand the problem: We need "
 'to find the probability of getting a sum of 11 when rolling a die three '
 "times.'}, {'step': 2, 't

In [8]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

reasoning_chain_teample = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            """Given Instruction, please generate {what}. Please use the following exampels.
    If reasoning and/or reasoning chains are provided, please use them as context to generate the {what}.
    Please only generate the {what} and do not include others.
    
    See the examples below:
    {examples}
    ---
    
    Instruction: {instruction}
    --
    {reasoning}
    --
    {reasoning_chains}   
    """,
        )
    ]
)


promt_example = reasoning_chain_teample.format(
    what="{what}",
    instruction="{instruction}",
    reasoning="{reasoning}",
    reasoning_chains="{reasoning_chains}",
    examples=reasoning_chain_examples,
)
print(promt_example)

Human: Given Instruction, please generate {what}. Please use the following exampels.
    If reasoning and/or reasoning chains are provided, please use them as context to generate the {what}.
    Please only generate the {what} and do not include others.
    
    See the examples below:
    ----
Example 1:

Instruction: If a die is rolled three times, what is the probability of getting a sum of 11? None

Reasoning: 1. Understand the problem: We need to find the probability of getting a sum of 11 when rolling a die three times.
2. Calculate total possible outcomes: A die has 6 faces, so for each roll, there are 6 possibilities. For three rolls, the total possible outcomes are 6^3 = 216.
3. Identify favorable outcomes: List all combinations of rolls that result in a sum of 11. There are 18 such combinations.
4. Calculate probability: Divide the number of favorable outcomes by the total possible outcomes: 18 / 216 = 1/12.
5. Conclusion: The probability of getting a sum of 11 when rolling a

In [9]:
llm = ChatUpstage(model="solar-pro")
chain = reasoning_chain_teample | llm | StrOutputParser()

In [10]:
instruction = "3.11 vs 3.9 which one is greater?"

In [11]:
reasoning = chain.invoke(
    {
        "instruction": instruction,
        "reasoning": "",
        "reasoning_chains": "",
        "what": "reasoning",
        "examples": reasoning_chain_examples,
    }
)

pprint(reasoning)

('Reasoning: 1. Identify the problem: We need to compare the numerical values '
 'of 3.11 and 3.9.\n'
 '2. Understand place values: Recognize that both numbers have the same whole '
 'number part (3), so we only need to compare the decimal parts (0.11 and '
 '0.9).\n'
 '3. Compare decimal parts: Since 0.9 is greater than 0.11, the number with '
 '0.9 as its decimal part is greater.\n'
 '4. Conclude: Based on the comparison, 3.9 is greater than 3.11.\n'
 '\n'
 "Reasoning Chains: [{'step': 1, 'thought': 'Identify the problem: We need to "
 "compare the numerical values of 3.11 and 3.9.'}, {'step': 2, 'thought': "
 "'Understand place values: Recognize that both numbers have the same whole "
 'number part (3), so we only need to compare the decimal parts (0.11 and '
 "0.9).'}, {'step': 3, 'thought': 'Compare decimal parts: Since 0.9 is greater "
 "than 0.11, the number with 0.9 as its decimal part is greater.'}, {'step': "
 "4, 'thought': 'Conclude: Based on the comparison, 3.9 is greater 

In [12]:
reasoning = chain.invoke(
    {
        "instruction": instruction,
        "reasoning": "",
        "reasoning_chains": "",
        "what": "reasoning",
        "examples": reasoning_chain_examples,
    }
)

pprint(reasoning)

('Reasoning: 1. Understand the problem: We need to compare two decimal '
 'numbers, 3.11 and 3.9, to determine which one is greater.\n'
 '2. Compare the whole number parts: Both numbers have the same whole number '
 'part, which is 3.\n'
 '3. Compare the decimal parts: Look at the first decimal place. For 3.11, '
 "it's 1, and for 3.9, it's 9. Since 9 is greater than 1, 3.9 is greater than "
 '3.11.\n'
 '4. Conclusion: 3.9 is greater than 3.11.\n'
 '\n'
 "Reasoning Chains: [{'step': 1, 'thought': 'Understand the problem: We need "
 'to compare two decimal numbers, 3.11 and 3.9, to determine which one is '
 "greater.'}, {'step': 2, 'thought': 'Compare the whole number parts: Both "
 "numbers have the same whole number part, which is 3.'}, {'step': 3, "
 "'thought': 'Compare the decimal parts: Look at the first decimal place. For "
 "3.11, it\\'s 1, and for 3.9, it\\'s 9. Since 9 is greater than 1, 3.9 is "
 "greater than 3.11.'}, {'step': 4, 'thought': 'Conclusion: 3.9 is greater "
 "th

In [13]:
reasoning_chain = chain.invoke(
    {
        "instruction": instruction,
        "reasoning": f"Reasoning: {reasoning}",
        "reasoning_chains": "",
        "what": "reasoning chain",
        "examples": reasoning_chain_examples,
    }
)

pprint(reasoning_chain)

('Reasoning: Reasoning: 1. Understand the problem: We need to compare two '
 'decimal numbers, 3.11 and 3.9, to determine which one is greater.\n'
 '2. Compare the whole number parts: Both numbers have the same whole number '
 'part, which is 3.\n'
 '3. Compare the decimal parts: Look at the first decimal place. For 3.11, '
 "it's 1, and for 3.9, it's 9. Since 9 is greater than 1, 3.9 is greater than "
 '3.11.\n'
 '4. Conclusion: 3.9 is greater than 3.11.\n'
 '\n'
 "Reasoning Chains: [{'step': 1, 'thought': 'Understand the problem: We need "
 'to compare two decimal numbers, 3.11 and 3.9, to determine which one is '
 "greater.'}, {'step': 2, 'thought': 'Compare the whole number parts: Both "
 "numbers have the same whole number part, which is 3.'}, {'step': 3, "
 "'thought': 'Compare the decimal parts: Look at the first decimal place. For "
 "3.11, it\\'s 1, and for 3.9, it\\'s 9. Since 9 is greater than 1, 3.9 is "
 "greater than 3.11.'}, {'step': 4, 'thought': 'Conclusion: 3.9 is gre

In [14]:
output = chain.invoke(
    {
        "instruction": instruction,
        "reasoning": f"Reasoning: {reasoning}",
        "reasoning_chains": f"Reasoning Chains: {reasoning_chain}",
        "examples": reasoning_chain_examples,
        "what": "Final answer of the instruction",
    }
)

pprint(output)

'3.9'
